In [437]:
import numpy as np
import random as rand
import pandas as pd
import math

In [438]:
# Função de Rosenbrock para cada par de dimensões consecutivas
def rosenbrock(x):
    total = 0

    # Iteração sobre as dimensões do vetor x
    for i in range(len(x) - 1):
        total += 100 * (x[i + 1] - x[i]**2)**2 + (1 - x[i])**2
    return total

In [439]:
class Particle:
    def __init__(self, x0, num_dimensions):
        self.position_i = [x for x in x0]
        self.velocity_i = [rand.uniform(-1, 1) for _ in range(num_dimensions)]
        self.pos_best_i = [x for x in x0]
        self.err_best_i = -1
        self.err_i      = -1

    def evaluate_particle(self, cost_func):
        self.err_i = cost_func(self.position_i)
        if self.err_i < self.err_best_i or self.err_best_i == -1:
            self.pos_best_i = [x for x in self.position_i]
            self.err_best_i = self.err_i

    def attVel(self, pos_best_g, w, c1, c2):

        for i in range(len(self.velocity_i)):
            r1 = rand.random()
            r2 = rand.random()

            vel_cognitive = c1 * r1 * (self.pos_best_i[i] - self.position_i[i])
            vel_social = c2 * r2 * (pos_best_g[i] - self.position_i[i])
            self.velocity_i[i] = w * self.velocity_i[i] + vel_cognitive + vel_social

    def attPos(self, bounds):
        for i in range(len(self.position_i)):
            self.position_i[i] = self.position_i[i] + self.velocity_i[i]

            if self.position_i[i] > bounds[i][1]:
                self.position_i[i] = bounds[i][1]

            if self.position_i[i] < bounds[i][0]:
                self.position_i[i] = bounds[i][0]

In [440]:
class PSO:
    def __init__(self, cost_func, x0, bounds, num_particles, max_iter, inercia, social, cognitivo, tol):
        num_dimensions = len(x0)
        err_best_g = math.inf
        pos_best_g = []
        swarm = [Particle(x0, num_dimensions) for _ in range(num_particles)]
        results = []

        for iteration in range(max_iter):
            for particle in swarm:
                particle.evaluate_particle(cost_func)
                if particle.err_i < err_best_g or math.isinf(err_best_g):
                    pos_best_g = list(particle.pos_best_i)
                    err_best_g = particle.err_i

            if err_best_g <= tol:
                results.append([pos_best_g, err_best_g])
                break

            for particle in swarm:
                particle.attVel(pos_best_g, inercia, social, cognitivo)
                particle.attPos(bounds)

            results.append([pos_best_g, err_best_g])

        self.results_df = pd.DataFrame(results, columns=['Best Position', 'Best Error'])

In [441]:
# blabla
inercia = 0.5
social = 1
cognitivo = 2
tol = 0.00001

# Parâmetros iniciais
initial = [5, 5]
bounds = [(-10, 10), (-10, 10)]
num_particles = 15
max_iterations = 9999

# Execução do PSO
pso_instance = PSO(rosenbrock, initial, bounds, num_particles, max_iterations, inercia, social, cognitivo, tol)

# Exibição dos resultados em forma de tabela
pso_instance.results_df

,Best Position,Best Error
0,"[5, 5]",40016.000000
1,"[4.524687568741913, 5.307028121170079]",23012.479795
2,"[3.6895867840438523, 5.351369533788171]",6832.771342
3,"[2.8152104218393523, 5.369144034841382]",656.744414
4,"[2.3446273765784222, 5.532420844275137]",1.931528
...,...,...
282,"[1.0212426729660122, 1.0455807907700043]",0.001150
283,"[1.0057224317395228, 1.0132332302302416]",0.000341
284,"[0.9996393872022084, 1.0000927611954906]",0.000066
285,"[0.9996393872022084, 1.0000927611954906]",0.000066
